<a href="https://colab.research.google.com/github/saotomryo/Use_MMOCR/blob/main/Use_MMOCR_seg_r31_1by16_fpnocr_toy_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import json
from skimage import measure
import numpy as np
print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device = ", device)

## 環境準備 MMCV MMOCRのインストール

In [ ]:
#!pip install mmcv-full==1.3.17
#!pip install mmcv-full
!pip install https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/mmcv_full-1.4.7-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
# Install mmdetection
!pip install mmdet

In [ ]:
HOME_PATH = "/content"

In [ ]:

!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -r requirements/build.txt
!pip install -v -e .  # or "python setup.py develop"

## コンフィグファイルの編集

In [ ]:
%cd /content/mmocr

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/content/mmocr/configs/textrecog/sar/sar_r31_parallel_decoder_toy_dataset.py')


## アノテーションファイルの変換

　toy_datasetのアノテーションファイルはファイル名（半角スペース)対象文字列で、テキストファイルとして作成します。

　ファイル名 対象の文字列

作成したテキストファイルをlmdbフォルダに形式変換して保存します。



In [ ]:
## アノテーションフォルダの設定
#　元になるテキストファイル
label_path = #ラベルファイルのパス
lmdb_path = #ラベル変換出力したファイルのパス

# 画像ファイルのパス
img_path = 画像フォルダのパス

# ルートフォルダ
root_path = ルートフォルダのパス

# 作業フォルダ（学習時のモデル出力先（エポック毎に出力))
work_path = 作業フォルダのパス


In [ ]:
!python /content/mmocr/tools/data/utils/txt2lmdb.py \
    -i $label_path \
    -o $lmdb_path

In [ ]:
from mmdet.apis import set_random_seed

# コンフィグを変更する
# sar

# 作業フォルダ
cfg.work_dir = work_path

# rootフォルダ
cfg.root = root_path

# アノテーションファイルのパス
cfg.train_anno_file1 = label_path
cfg.train_anno_file2 = lmdb_path
cfg.img_prefix = img_path
cfg.train1.ann_file = label_path
cfg.train1.img_prefix = img_path
cfg.train2.ann_file = label_path
cfg.train2.img_prefix = img_path
cfg.test.ann_file = lmdb_path
cfg.test.img_prefix = img_path
cfg.test_anno_file1 = lmdb_path
cfg.train_list[0].ann_file = label_path
cfg.train_list[0].img_prefix = img_path
cfg.train_list[1].ann_file = lmdb_path
cfg.train_list[1].img_prefix = img_path
cfg.test_list[0].ann_file = lmdb_path
cfg.test_list[0].img_prefix = img_path
cfg.data.test.datasets[0].ann_file = lmdb_path
cfg.data.test.datasets[0].img_prefix = img_path
cfg.data.train.datasets[0].ann_file = label_path
cfg.data.train.datasets[0].img_prefix = img_path
cfg.data.train.datasets[1].ann_file = lmdb_path
cfg.data.train.datasets[1].img_prefix = img_path
cfg.data.val.datasets[0].ann_file = lmdb_path
cfg.data.val.datasets[0].img_prefix = img_path


# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.001 / 8
cfg.lr_config.warmup = None
# Choose to log training results every 40 images to reduce the size of log file. 
cfg.log_config.interval = 40

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


# 編集したコンフィグをファイル出力
cfg.dump('/content/seg_r31_1by16_fpnocr_toy_dataset.py')


In [ ]:
from mmocr.datasets import build_dataset
from mmocr.models import build_detector
from mmocr.apis import train_detector
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
#mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

## 学習結果の確認

In [ ]:
%cd /content/mmocr

In [ ]:

from mmocr.apis import init_detector, model_inference

img = #確認する画像ファイルのパス
checkpoint = # 利用するモデルのパス
out_file = # 出力するファイルのパス

model = init_detector(cfg, checkpoint, device=device)
if model.cfg.data.test['type'] == 'ConcatDataset':
    model.cfg.data.test.pipeline = model.cfg.data.test['datasets'][0].pipeline

result = model_inference(model, img)
print(f'result: {result}')

img = model.show_result(
        img, result, out_file=out_file, show=False)

